In [1]:
import multiprocessing
from multiprocessing.queues import Queue
from scrapy import  signals 
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from scrapy.utils.project import get_project_settings

class CrawlerWorker(multiprocessing.Process):
    def __init__(self, result_queue, spider, settings=None):
        multiprocessing.Process.__init__(self)
        self.settings = settings or get_project_settings()
        self.result_queue = result_queue
        self.spider = spider
        self.items = []
        dispatcher.connect(self._item_scraped, signals.item_scraped)

    def _item_scraped(self, item):
        self.items.append(item)

    def run(self):
        self.crawler_process = CrawlerProcess(self.settings)
        crawler = self.crawler_process.create_crawler(self.spider)
        self.crawler_process.crawl(crawler)
        self.crawler_process.start()
        self.result_queue.put(self.items)
        self.crawler_process.stop()

def crawl():
    result_queue = Queue()
    crawler = CrawlerWorker(result_queue, "cncb")
    crawler.start()
    return result_queue.get()

/Users/chibs/.conda/envs/ml-learning/lib/python2.7/site-packages/ipykernel/__main__.py:5: ScrapyDeprecationWarning: Importing from scrapy.xlib.pydispatch is deprecated and will no longer be supported in future Scrapy versions. If you just want to connect signals use the from_crawler class method, otherwise import pydispatch directly if needed. See: https://github.com/scrapy/scrapy/issues/1762


In [5]:
result = crawl()
import pandas as pd
df = pd.DataFrame(map(dict,result),columns=['category', 'question', 'answer', 'language'])
df.to_csv('cncb-qna.csv',encoding='utf-8',index=False)